# 📘 Flotorch Email Ticket Router Demo (LangGraph)

This notebook demonstrates how to automate email-based ticket routing using Flotorch Agents with LangGraph framework and custom tools for Gmail.

## 🧭 Workflow Overview

1. **Fetch unread emails** from your inbox
2. **The agent** (configured in Flotorch Console) analyzes each email:
   - Detects intent (bug, feature, issue, etc.)
   - Determines routing team & priority
   - Summarizes the issue clearly
3. **Automatically routes** the task to the correct team lead
4. **Sends acknowledgment** to the sender confirming receipt

## 💡 Goal

Streamline internal task management from **"email → assignment"** in one autonomous flow.


# 📋 PRE-RUN CHECKLIST

Before you run this notebook, complete these setup steps:

## 1️⃣ Create a Flotorch Agent in Console

- **Agent name**: `support-ticket-inbox-router` (or match `AGENT_NAME` variable)
- **System Prompt** (⚠️ **IMPORTANT**: All curly braces must be escaped with double braces `{{}}` for LangGraph):
      You are the Support Ticket Inbox Router.

      Workflow:

      1) Call fetch_mail_tool() - which returns a dictionary with 'status', 'emails', and 'message' keys.
         - If emails found: {{"status": "success", "emails": [{{"id": "...", "sender": "...", "subject": "...", "body": "..."}}, ...], "message": "Found N email(s) to process."}}
         - If no emails found: {{"status": "no_emails", "emails": [], "message": "No new emails found matching the criteria."}}

      2) Check the 'status' field:
         - If status is "no_emails", return a final message: "No new emails found matching the criteria. No action needed."
         - If status is "success", proceed to process the emails in the 'emails' list

      3) For each email in the 'emails' list:
         a) Analyze subject+body and produce a structured JSON with keys:
            - intent: one of ["bug","feature","task","issue","query"]
            - title: short developer-friendly title
            - summary: 1-3 sentence summary with reproduction steps if provided
            - routing_team: one of ["ui","python_backend","views_backend","devops","qa","technical"]
            - priority: one of ["P0 - Critical","P1 - High","P2 - Medium","P3 - Low","P4 - Info"]
            - recommended_sla: e.g. "Immediate (<30m)", "1 hour", "4 hours", "24 hours"
            - ack_message: short text to send to sender
            - notify_message: short text to send to lead

         b) Call route_task_tool(routing_team, title, summary, priority)
            - expect result: {{"lead_name","lead_email","status"}}

         c) Call send_ack_tool(sender, ack_message, ack_subject) to acknowledge sender.

      4) After processing all emails, return a summary JSON list of processed items with their tool-call statuses.

      **CRITICAL**: Always check the 'status' field from fetch_mail_tool() response:
      - If status is "no_emails", immediately return a final message: "No new emails found matching the criteria. No action needed."
      - Do NOT call route_task_tool or send_ack_tool if status is "no_emails"
      - Only process emails when status is "success"

      Important: Only call the tools above. Do not send emails yourself — use send_ack_tool and route_task_tool to send messages. Be deterministic and concise.
      **STRICTLY CALL ALL TOOLS** for correct working flow only when emails are found (status is "success").
      
- **Goal**: 
      To automatically manage and route internal task or issue requests received via email.
      When executed, this agent:
      1. Fetches new emails from a configured inbox using the fetch_mail tool.
      2. Analyzes each email's subject and body to identify its intent (bug, feature, issue, or task), summarize the content, and determine the responsible team.
      3. Uses reasoning to decide the priority and urgency of each item.
      4. Invokes tools to route the task to the correct team lead (via email or future task system integration).
      5. Sends an acknowledgment reply to the original sender confirming the action taken.
      The agent ensures that communication between the sender and the responsible team is automated and consistent.

- **Model Settings**: add llm model
- **Save/Deploy**: Save and publish the agent

## 2️⃣ Create and Store Your Flotorch Credentials

In the notebook **Configuration** cell, set:

```python
FLOTORCH_API_KEY = "sk_..."
FLOTORCH_BASE_URL = "https://<your-gateway>.flotorch.cloud"
```

## 3️⃣ Prepare Gmail (App Password + IMAP)

### Steps:

1. **Sign in** to your Gmail account
2. **Enable 2-Step Verification** (if not already enabled)
3. **Create an App Password**:
   - Go to **Google Account → Security → App passwords**
   - Select **App**: Mail, **Device**: Other, name (e.g., "Notebook")
   - Copy the 16-character app password
4. **Enable IMAP** in Gmail settings:
   - Gmail → Settings → See all settings → Forwarding and POP/IMAP → **Enable IMAP**

### In the notebook Configuration cell, set:

```python
GMAIL_EMAIL = "your_email@gmail.com"
GMAIL_APP_PASSWORD = "16-char-app-password"
```

## 4️⃣ Customize Team Leads

In the notebook `TEAMS` dict, set correct `lead_emails` for your organization.

## refer last cell for sample examples do a test mail with that sample


In [ ]:
# install flotorch langgraph package
%pip install flotorch[langgraph]==2.4.0b1


In [ ]:
# Allow kernel to run multiple async operations
import nest_asyncio
nest_asyncio.apply()

# ======================================================================
# 🧩 STEP 2 — Configuration
# ======================================================================
# Fill in your credentials directly below.
# This keeps the notebook self-contained and demo-ready.
# ======================================================================

FLOTORCH_API_KEY = "sk_"         # 🔑 your Flotorch API key
FLOTORCH_BASE_URL = "<gateway url>"   # 🌐 your Flotorch Gateway URL
AGENT_NAME = "<agent_name>"       # 🤖 Agent name configured in Console
APP_NAME = "email_ticket_router_demo"
USER_ID = "email_router_user_001"

# Gmail Credentials (create an App Password under Google Account -> Security -> App Passwords)
GMAIL_EMAIL = "<gmail>"
GMAIL_APP_PASSWORD = "<app password>"

# Team routing map (can customize for your org)
TEAMS = {
    "ui": {"lead_name": "<name>", "lead_email": "<gmail>"},
    "python_backend": {"lead_name": "<name>", "lead_email": "<gmail>"},
    "devops": {"lead_name": "<name>", "lead_email": "<gmail>"},
}

# Mail settings
IMAP_SERVER = "imap.gmail.com"
SMTP_SERVER = "smtp.gmail.com"
MAX_EMAILS_PER_RUN = 5
GMAIL_SUBJECT_KEYWORDS = ["bug", "issue", "feature", "task"]

print("✅ Configuration loaded successfully.")


In [ ]:
# ======================================================================
# 📬 STEP 3 — Gmail Utilities
# ======================================================================
# Functions for connecting to Gmail (IMAP & SMTP),
# fetching unread emails, and sending replies.
# ======================================================================

import imaplib, smtplib, email, re
from email.header import decode_header, make_header
from email.mime.text import MIMEText
from email.utils import parseaddr
from typing import List, Tuple

# --- IMAP Connection ---
def connect_imap():
    m = imaplib.IMAP4_SSL(IMAP_SERVER)
    m.login(GMAIL_EMAIL, GMAIL_APP_PASSWORD)
    return m

# --- Fetch unread emails matching keywords ---
def search_unseen_matching_subject(m, keywords: List[str], max_count: int):
    m.select("INBOX")
    status, data = m.search(None, 'UNSEEN')
    if status != 'OK':
        return []
    ids = data[0].split()
    results = []
    for msg_id in reversed(ids):
        if len(results) >= max_count:
            break
        status, msg_data = m.fetch(msg_id, '(RFC822)')
        if status != 'OK':
            continue
        msg = email.message_from_bytes(msg_data[0][1])
        subj_raw = msg.get('Subject', '')
        try:
            subj = str(make_header(decode_header(subj_raw)))
        except Exception:
            subj = subj_raw
        if any(k.lower() in subj.lower() for k in keywords):
            results.append((msg_id, msg))
    return results

# --- Extract sender, subject, body ---
def extract_sender_subject_body(msg) -> Tuple[str, str, str]:
    sender = parseaddr(msg.get('From', ''))[1]
    subject = str(make_header(decode_header(msg.get('Subject', ''))))
    body = ""
    if msg.is_multipart():
        for part in msg.walk():
            if part.get_content_type() == 'text/plain' and 'attachment' not in str(part.get("Content-Disposition", "")):
                body = part.get_payload(decode=True).decode(part.get_content_charset() or 'utf-8', errors='ignore')
                break
    else:
        payload = msg.get_payload(decode=True)
        if payload:
            body = payload.decode(msg.get_content_charset() or 'utf-8', errors='ignore')
    body = re.sub(r"^>+.*$", "", body, flags=re.MULTILINE).strip()
    return sender, subject, body

# --- SMTP Send Helper ---
def send_smtp_text(to_addr: str, subject: str, body: str):
    msg = MIMEText(body, _charset='utf-8')
    msg['From'] = GMAIL_EMAIL
    msg['To'] = to_addr
    msg['Subject'] = subject
    with smtplib.SMTP_SSL(SMTP_SERVER, 465) as s:
        s.login(GMAIL_EMAIL, GMAIL_APP_PASSWORD)
        s.sendmail(GMAIL_EMAIL, [to_addr], msg.as_string())

print("📨 Gmail utilities ready.")


In [ ]:
# ======================================================================
# 🛠️ STEP 4 — Define Custom Tools
# ======================================================================
# These tools are the interface between the agent and real-world actions.
# The agent decides *when and how* to call them.
# ======================================================================

from langchain.agents import tool
from typing import Dict, Optional, List, Any

# --- Tool A: Fetch unread emails ---
@tool
def fetch_mail_tool(max_results: int = MAX_EMAILS_PER_RUN) -> Dict[str, Any]:
    """
    Fetch unread emails from the inbox that match subject keywords.
    
    Args:
        max_results: Maximum number of emails to fetch (default: MAX_EMAILS_PER_RUN)
    
    Returns:
        Dictionary with 'status' and 'emails' keys. If emails found, 'emails' contains list of email dicts.
        If no emails found, 'status' is 'no_emails' and 'emails' is empty list.
        Format: {"status": "success" or "no_emails", "emails": [{"id": "...", "sender": "...", "subject": "...", "body": "..."}, ...]}
    """
    m = connect_imap()
    matches = search_unseen_matching_subject(m, GMAIL_SUBJECT_KEYWORDS, max_results)
    emails = []
    for msg_id, msg in matches:
        sender, subject, body = extract_sender_subject_body(msg)
        emails.append({
            "id": msg_id.decode() if isinstance(msg_id, bytes) else str(msg_id),
            "sender": sender,
            "subject": subject,
            "body": body
        })
    m.logout()
    
    # Always return a structured response with status
    if emails:
        return {"status": "success", "emails": emails, "message": f"Found {len(emails)} email(s) to process."}
    else:
        return {"status": "no_emails", "emails": [], "message": "No new emails found matching the criteria."}

# --- Tool B: Route the task to the appropriate team lead ---
@tool
def route_task_tool(routing_team: str, title: str, summary: str, priority: str) -> Dict[str, str]:
    """
    Route a task to the appropriate team lead via email.
    
    Args:
        routing_team: Team name (e.g., 'ui', 'python_backend', 'devops')
        title: Short title for the task
        summary: Detailed summary of the task
        priority: Priority level (e.g., 'P0 - Critical', 'P1 - High', etc.)
    
    Returns:
        Dictionary with lead_name, lead_email, and status
    """
    team_info = TEAMS.get(routing_team.lower())
    if not team_info:
        return {"status": f"error: unknown team '{routing_team}'"}
    subject = f"[{priority}] {title}"
    body = f"Hi {team_info['lead_name']},\n\nAssigned: {title}\nPriority: {priority}\n\nSummary:\n{summary}\n\n— Task Router"
    send_smtp_text(team_info["lead_email"], subject, body)
    return {"lead_name": team_info["lead_name"], "lead_email": team_info["lead_email"], "status": "sent"}

# --- Tool C: Send acknowledgment to sender ---
@tool
def send_ack_tool(sender_email: str, ack_body: str, ack_subject: Optional[str] = None) -> Dict[str, str]:
    """
    Send an acknowledgment email to the original sender.
    
    Args:
        sender_email: Email address of the original sender
        ack_body: Message body for the acknowledgment
        ack_subject: Optional subject line (default: 'Acknowledgment: we've received your request')
    
    Returns:
        Dictionary with 'to' and 'status' keys
    """
    subject = ack_subject or "Acknowledgment: we've received your request"
    send_smtp_text(sender_email, subject, ack_body)
    return {"to": sender_email, "status": "sent"}

# Wrap as LangGraph-compatible tools list
custom_tools = [fetch_mail_tool, route_task_tool, send_ack_tool]

print("🧰 Custom tools registered:", [tool.name for tool in custom_tools])


In [ ]:
# ======================================================================
# ⚙️ STEP 5 — Initialize Flotorch LangGraph Agent
# ======================================================================
# Connect to your configured agent in Flotorch Console.
# Make sure your Console agent has the correct system prompt and goal.
# ======================================================================

from flotorch.langgraph.agent import FlotorchLangGraphAgent
from flotorch.langgraph.sessions import FlotorchLanggraphSession

# Create session checkpointer for conversational context (optional)
checkpointer = FlotorchLanggraphSession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    app_name=APP_NAME,
    user_id=USER_ID
)

# Initialize Flotorch LangGraph Agent with custom tools
flotorch_client = FlotorchLangGraphAgent(
    agent_name=AGENT_NAME,
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    custom_tools=custom_tools,   # register your local action tools
    checkpointer=checkpointer    # enable session memory
)

agent = flotorch_client.get_agent()

# Configuration for session management
config = {"configurable": {"thread_id": "email_router_thread_001"}}

print("Connected to Flotorch Agent:", AGENT_NAME)
print(f"✅ LangGraph agent initialized with {len(custom_tools)} custom tools")


In [ ]:
# ======================================================================
# ▶️ STEP 6 — Run Agent (check below cell for sample example of mail before running agent)
# ======================================================================
# This section executes the agent once.
# The agent will:
#   - Fetch unread emails
#   - Analyze & classify them
#   - Route tasks to leads
#   - Send acknowledgments
# ======================================================================

def run_agent_once():
    """
    Ask the agent (configured in Console) to fetch emails and process them.
    The Console system prompt should instruct the agent to:
      - call fetch_mail_tool() to get list of emails
      - if emails are found, for each email, analyze and output structured JSON:
            {intent, title, summary, routing_team, priority, recommended_sla, ack_message, notify_message}
      - call route_task_tool(routing_team, title, summary, priority)
      - call send_ack_tool(sender, ack_message, ack_subject)
      - if no emails are found, return a message indicating no emails were found
      - Return a short JSON summary
    """
    # The text prompt just triggers the agent flow; the heavy instructions live in Console
    task = "Process unread mailbox: fetch and handle matching messages. If no emails are found, inform me that no new emails were found."
    
    try:
        # LangGraph agent automatically handles multiple tool calls in sequence
        # It will continue until the agent provides a final response
        response = agent.invoke({"messages": task}, config)
        
        # Extract and display the final response
        # LangGraph returns messages in the response
        if 'messages' in response:
            final_message = response['messages'][-1]
            if hasattr(final_message, 'content'):
                output = final_message.content
            elif hasattr(final_message, 'text'):
                output = final_message.text
            else:
                output = str(final_message)
        else:
            output = str(response)
        
        print("\n--- Agent final output ---\n", output)
        
        return response
    except Exception as e:
        print(f"\n❌ Error occurred: {e}")
        
        

# Run the agent
run_agent_once()


In [ ]:
# ======================================================================
# ▶️ Sample email examples for the agent to process
# ======================================================================
#sample1:Feature Request (UI)
'''
Subject: Feature: Add confirmation pop-up for experiment deletion

Body:
Hi Team,

When users delete an experiment there is no confirmation popup. Please implement a confirmation dialog on the UI to avoid accidental deletes.

Steps to reproduce:
1) Open experiment list.
2) Click delete on an experiment.
3) Observe deletion happens immediately.

Thanks,
QA
'''

#sample2:Bug Report (Backend)
'''
Subject: Bug: API returns 500 on /v1/experiments

Body:

Hi,

The experiments list API intermittently returns 500 errors. This started after the last deployment.

Error: InternalServerError stack trace...
Affects production for some users.

Please investigate urgently.
'''
